In [20]:
import numpy as np
import emcee
from joblib import dump, load
import os

from sklearn.gaussian_process import GaussianProcessRegressor as GPR

In [12]:
cwd = os.getcwd()

In [15]:
#emulator models (stand in for actual GAMMA model)
#4 sec to load in
stellar_mass_em = load(r'C:\\Users\\carli\\Documents\\Astrophysics General\\iCER ACRES\\GAMMA-EM\\stellar_mass_emulator.joblib')
metallicity_emulator_em = load(r'C:\\Users\\carli\\Documents\\Astrophysics General\\iCER ACRES\\GAMMA-EM\\metallicity_emulator.joblib')

In [ ]:
#log-likelihood function
#recieve individual variable parameters
#put them in an array
#feed that into the prediction model
#get back predicted results and N by N covariance matrix (yay sklearn gives back the right one)
#calculate log_prob between APOGEE observation and model results

def log_prob():
    return 

In [27]:
x = np.load(r'C:\\Users\\carli\\Documents\\Astrophysics General\\iCER ACRES\\GAMMA-EM\\sample_sets\\em_sample_points200.npy')
true_y = np.load(r'C:\\Users\\carli\\Documents\\Astrophysics General\\iCER ACRES\\GAMMA-EM\\sample_sets\\gal_Mstar_200.npy')

In [37]:
print(stellar_mass_em[0].score(x,np.log10(true_y[:,0])))
#proof the loaded in emulators work

0.8886051428517535


In [40]:
#parameters to fit - eventually should pull this from the GAMMA-EM sampling script
#these starting values are just about the average of the current proposed range of values
#t_ff_index, f_t_ff, sfe_m_index, sfe, f_dyn, t_sf_z_dep, exp_ml, mass_loading, f_halo_to_gal_out, nb_1a_per_m
start = np.array([1.0,5.5,0.1,0.055,0.3,1.0,1.0,1.0,0.5, 1.0e-3])
t_ff_index = 1.0
f_t_ff = 5.5
sfe_m_index = 0.1
sfe = 0.055
f_dyn = 0.3
t_sf_z_dep = 1.0
exp_ml = 1.0
mass_loading = 1.0
f_halo_to_gal_out = 0.5
nb_1a_per_m = 1.0e-3

[1.0e+00 5.5e+00 1.0e-01 5.5e-02 3.0e-01 1.0e+00 1.0e+00 1.0e+00 5.0e-01
 1.0e-03]


In [46]:
dwarf_Mstar = np.loadtxt(fname="C:\Users\carli\Documents\Astrophysics General\iCER ACRES\Observations\McConnachie Paper\NearbyGalaxies.dat")

ValueError: could not convert string to float: 'This'

In [43]:
#number of parameters and walkers
ndim, nwalkers = 10, 10e10